## Starting from an already performed simulation result (not run in this Python instance)

.log file 지우면 안 됨.  한 directory에는 하나의 run만. 

In [1]:
#from pts.do.prompt import do
#%matplotlib inline
#%matplotlib notebook
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
from matplotlib import rcParams
import matplotlib.pyplot as plt
import logging # This is a Python standard package.
import pts.simulation as sm
import pts.utils as ut
import pts.visual as vis

from glob import glob
from PIL import Image

import pyram

In [2]:
#redundant = np.genfromtxt("../list_aexprestart_nout.txt", dtype=[('nout','int'),('zred','float')])
#good_nouts = np.setdiff1d(np.arange(900), redundant["nout"])

## RGB image

Looking at pts/visual/do/make_images.py  
Two use cases:
1. Specify wavelength  
2. Specify bands  

Q: Any support for SB limit??


Decades not applied to arcsinh? 

In [11]:
import pts.band as bnd
wavelengths = None

if wavelengths is not None:
    tuples = { name: wavelengths << sm.unit("micron") }
    for sim in sims:
        vis.makeRGBImages(sim, wavelengthTuples=tuples, fileType=type)


target_nouts=[169, 174, 179, 183,
              188, 193, 198, 202,
              206, 211, 216, 221]
good_nouts=[110, 158, 169, 174, 179, 
            183, 188, 193, 198, 202, 206,
            208, 211, 216, 222, 232, 237, 
            242, 247, 300, 446]

output_type = "total"
quality = ["intermediate", "high"][0]

if quality == 'intermediate':
    name = "SDSS_RGB_LQ"
    prefix = lambda nout: f"g13_{nout}"
    decs=[3.5,4,4.5,5,5.5]
elif quality == "high":
    name = "SDSS_RGB_HQ"
    prefix = lambda nout: f"g13_{nout}_HQ"
    decs=[3.5,4,4.5]

# Keep flux range the same throughout snapshots.z`
fmin_ash, fmax_ash =  2e-3, 2e2
fmin_log1, fmax_log1 = None, None
fmin_log2, fmax_log2 = 5e-3, 5e2
fmin_log3, fmax_log3 = None, None

all_fns=[]

#for nout in target_nouts[]:
for nout in [206]:
    # try bands
    #colors = "SDSS_Z,SDSS_G,SDSS_U"#,
    colors = "SDSS_I,SDSS_G,SDSS_U" #MASS_2MASS_H,2MASS_2MASS_J,2MASS_2MASS_KS"
    segments = colors.split(',')
    if len(segments) != 3:
        raise ut.UserError("colors argument must have three comma-separated segments")

    try: bands = [ bnd.BroadBand(segment) for segment in segments ]
    except ValueError: bands = None
    
    
    print("Generating mock images... nout=", nout)
    repo = f"./01000/{nout:05d}/"#faceon_redshift_"
    # load values from JP's value

    sim = sm.createSimulation(outDirPath=repo, prefix=prefix(nout))
    skifile = sim.parameters()
    
    #for inst in sim.instruments():
    inst = sim.instruments()[0]
    totalfluxpath = inst.outFilePaths(fileType="total.fits")[0]
    datacube = sm.loadFits(totalfluxpath) # loadFits & getFitsAxes return 'astropy' quantities with units attached.
    x,y,wavelengths = sm.getFitsAxes(totalfluxpath)


    if bands is not None:
        # 0.6, 0.75, 1.2
        contributions = [ (bands[0], 0.6, 0, 0), (bands[1], 0, 0.75, 0), (bands[2], 0, 0, 1.4) ]
        # Could loop over sims
        # Make RGB images of ALL instruments.
        #fmax_ash, fn_ash = vis.makeConvolvedRGBImages(sim, 
        #                                                contributions=contributions, 
        #                                                fileType=output_type, 
        #                                                name=name,
        #                                                stretch=np.arcsinh, 
        #                                                decades=5, 
        #                                                fmin=fmin_ash, 
        #                                                fmax=fmax_ash)
        print("arcsinh", fmin_ash, fmax_ash)

        fmax_log0, fns_log0 = vis.makeConvolvedRGBImages(sim, 
                                                contributions=contributions, 
                                                fileType=output_type, 
                                                name=name,
                                                stretch='log', 
                                                decades=decs[0],
                                                fmin=fmin_log1, 
                                                fmax=fmax_log1)
        fmax_log1, fns_log1 = vis.makeConvolvedRGBImages(sim, 
                                                contributions=contributions, 
                                                fileType=output_type, 
                                                name=name,
                                                stretch='log', 
                                                decades=decs[1],
                                                fmin=fmin_log2, 
                                                fmax=fmax_log2)
        #fmin_log2, fmax_log2, fns_log2 = vis.makeConvolvedRGBImages(sim, 
        #                                        contributions=contributions, 
        #                                        fileType=output_type, 
        #                                        name=name,
        #                                        stretch='log', 
        #                                        decades=decs[2],
        #                                        fmin=fmin_log3, 
        #                                        fmax=fmax_log3,
        #                                        return_fn=True)
        #print(dec, fmin_log3, fmax_log3)
        #all_fns.append(fns_log2)

Generating mock images... nout= 206
arcsinh 0.002 200.0


## Merge stamps

In [19]:
im.histogram

In [20]:
from PIL import ImageOps

In [36]:
rotate = False

fig, axs = plt.subplots(3,4)
fig.set_size_inches(8,6)
axs = axs.ravel()


ori ='faceon'
getfn = lambda nout: f"/home/hoseung/Work/data/NH/JP/01000/00{nout}/g13_{nout}_{ori}_total_SDSS_RGB_LQ_log_dec5.0.png"

target_nouts=[169, 174, 179, 183,
              188, 193, 198, 202,
              206, 211, 216, 221]

crop_dx = 250
npix=1000
normalize=True

for ax, nout in zip(axs,target_nouts):
    fn = getfn(nout)
    proj = fn.split("_total_")[0][-6:]
    im = Image.open(fn)
    if rotate:
        im = im.rotate(80)
    if normalize:
        ImageOps.autocontrast(im, cutoff=5)
    im = im.crop((crop_dx,crop_dx,npix-crop_dx,npix-crop_dx))
    #new_img_size = int((npix-2*crop_dx)/1)
    #im = im.resize((new_img_size,new_img_size), resample=Image.BILINEAR)
    
    ax.imshow(im)
    ax.set_yticks(())
    ax.set_xticks(())
    
    s = pyram.load.sim.Sim(nout=nout, base='../')
    if nout == target_nouts[0]:
        t0 = s.info.tGyr
        ax.text(0.05, 0.9, "z={:.2f}".format(s.info.zred), color='w', transform=ax.transAxes)
    elif nout == target_nouts[-1]:
        ax.text(0.05, 0.9, "+{:.2}Gyr        z={:.2f}".format(s.info.tGyr - t0, s.info.zred),
                color='w', transform=ax.transAxes)
    else:
        ax.text(0.05, 0.9, "+{:.2}Gyr".format(s.info.tGyr - t0), color='w', transform=ax.transAxes)
                
fig.subplots_adjust(left=0.1, right=0.97, top=0.97,bottom=0.1, wspace=0.01, hspace = .01)

# turn off ticks
axs = axs.reshape(3,4)
for ax in axs[:,0]:
    ax.set_yticks((50,175,300)) # 1000 : 60 -> new_im....
    ax.set_yticklabels(["15", "0", "-15"])

for ax in axs[2,:]:
    ax.set_xticks((50,175,300))
    ax.set_xticklabels(["-15", "0", "15"])

fig.text(0.54, 0.04, 'kpc', va='center', ha='center', fontsize=rcParams['axes.labelsize'])
fig.text(0.04, 0.54, 'kpc', va='center', ha='center', rotation='vertical', fontsize=rcParams['axes.labelsize'])
plt.savefig("sequence_" + proj + fn.split(".png")[0][-11:-1]+"UGI.pdf", dpi=300)
plt.close()

Age of the universe (now/z=0): -3.114 / 0.000 Gyr, z = 2.84816
Age of the universe (now/z=0): -3.114 / 0.000 Gyr, z = 2.84816
Simulation set up.
Age of the universe (now/z=0): -3.051 / 0.000 Gyr, z = 2.78239
Age of the universe (now/z=0): -3.051 / 0.000 Gyr, z = 2.78239
Simulation set up.
Age of the universe (now/z=0): -2.990 / 0.000 Gyr, z = 2.71959
Age of the universe (now/z=0): -2.990 / 0.000 Gyr, z = 2.71959
Simulation set up.
Age of the universe (now/z=0): -2.931 / 0.000 Gyr, z = 2.65899
Age of the universe (now/z=0): -2.931 / 0.000 Gyr, z = 2.65899
Simulation set up.
Age of the universe (now/z=0): -2.874 / 0.000 Gyr, z = 2.60104
Age of the universe (now/z=0): -2.874 / 0.000 Gyr, z = 2.60104
Simulation set up.
Age of the universe (now/z=0): -2.819 / 0.000 Gyr, z = 2.54501
Age of the universe (now/z=0): -2.819 / 0.000 Gyr, z = 2.54501
Simulation set up.
Age of the universe (now/z=0): -2.765 / 0.000 Gyr, z = 2.49096
Age of the universe (now/z=0): -2.765 / 0.000 Gyr, z = 2.49096
Simu

Smoothing?

In [4]:
# SED plot
micron = sm.unit("micron")
vis.plotSeds(sim, decades=4, figSize=(7,5), outFileName="try1_sed.png")